```
pip install --pre pystan
```

In [1]:
import nest_asyncio; nest_asyncio.apply()
import stan

import numpy as np

import jax

import numpyro; numpyro.enable_x64()
from numpyro.distributions import constraints, biject_to
from numpyro.infer import MCMC, NUTS, init_to_value

from age_model import model
from data import get_data, transform_data, generate_init_values

In [2]:
data = get_data()
transformed_data = transform_data(data)
kernel = NUTS(model, init_strategy=init_to_value(values=generate_init_values(data, seed=0)))
mcmc = MCMC(kernel, num_warmup=0, num_samples=1)
mcmc.run(np.zeros(2, dtype=np.uint32), transformed_data, reparam=False)
z = {k: v[0] for k, v in mcmc.get_samples().items()}

sample: 100%|██████████| 1/1 [00:35<00:00, 35.95s/it, 1 steps of size 1.00e+00. acc. prob=0.00]


In [3]:
with open("stan_model_main.stan") as f:
    stan_code = f.read()
stan_model = stan.build(stan_code, data=data, random_seed=1)
stan_fit = stan_model.sample(num_chains=1, num_warmup=20, stepsize=1e-6, max_depth=5, num_samples=2)

Building...
Messages from stanc:
  The parameter impact_intv_children_effect has no priors.
  The parameter log_ifr_age_base has no priors.
  The parameter log_relsusceptibility_age_reduced has 2 priors.
  The parameter upswing_timeeff_reduced has 2 priors.
  Your Stan program has a parameter impact_intv_children_effect with a lower
  and upper bound in its declaration. These hard constraints are not
  recommended, for two reasons: (a) Except when there are logical or physical
  constraints, it is very unusual for you to be sure that a parameter will
  fall inside a specified range, and (b) The infinite gradient induced by a
  hard constraint can cause difficulties for Stan's sampling algorithm. As a
  consequence, we recommend soft constraints rather than hard constraints;
  for example, instead of constraining an elasticity parameter to fall
  between 0, and 1, leave it unconstrained and give it a normal(0.5,0.5)
  prior distribution.
Warning at '/tmp/httpstan_z92ao6qb/model_xh4v2tcz

In [4]:
params_support = {
  "R0": constraints.positive,
  "e_cases_N0": constraints.positive,
  "sd_dip_rnde": constraints.positive,
  "phi": constraints.positive,
  "log_ifr_age_base": constraints.less_than(0.),
  "hyper_log_ifr_age_rnde_mid1": constraints.positive,
  "hyper_log_ifr_age_rnde_mid2": constraints.positive,
  "hyper_log_ifr_age_rnde_old": constraints.positive,
  "log_ifr_age_rnde_mid1": constraints.positive,
  "log_ifr_age_rnde_mid2": constraints.positive,
  "log_ifr_age_rnde_old": constraints.positive,
  "upswing_timeeff_reduced": constraints.positive,
  "sd_upswing_timeeff_reduced": constraints.positive,
  "hyper_timeeff_shift_mid1": constraints.positive,
  "timeeff_shift_mid1": constraints.positive,
  "impact_intv_children_effect": constraints.interval(.1, 1.),
  "impact_intv_onlychildren_effect": constraints.positive
}

In [5]:
params = stan_fit.to_frame().to_dict()
z0 = {}
z1 = {}
for k in stan_fit.param_names:
    if k not in z:
        continue
    z0[k] = np.array([v[0] for k1, v in params.items() if k1.startswith(k)])
    z1[k] = np.array([v[1] for k1, v in params.items() if k1.startswith(k)])
    shape = z[k].shape
    if len(shape) == 2:
        z0[k] = z0[k].reshape((shape[1], shape[0])).T
        z1[k] = z1[k].reshape((shape[1], shape[0])).T
    elif len(shape) == 0:
        z0[k] = z0[k].reshape(())
        z1[k] = z1[k].reshape(())
    if k in params_support:
        z0[k] = biject_to(params_support[k]).inv(z0[k])
        z1[k] = biject_to(params_support[k]).inv(z1[k])
pe0 = -params["lp__"][0]
pe1 = -params["lp__"][1]
print("stan pe0 and pe1:", pe0, pe1)
print("stan pe diff:", pe1 - pe0)
potential_fn = mcmc.sampler._potential_fn_gen(transformed_data)
print("numpyro pe0 and pe1:", potential_fn(z0), potential_fn(z1))
print("numpyro pe diff:", potential_fn(z1) - potential_fn(z0))

stan pe0 and pe1: 1547775.09556057 1547766.3554903092
stan pe diff: -8.740070260828361
numpyro pe0 and pe1: 1547775.0955605675 1547766.3554903062
numpyro pe diff: -8.740070261294022


In [6]:
pe_and_grad = jax.jit(jax.value_and_grad(potential_fn))
print(pe_and_grad(z1)[0])

1547766.3554904326


In [7]:
%%time
a, b = pe_and_grad(z0)
print(a)

1547775.0955604939
CPU times: user 188 ms, sys: 101 ms, total: 289 ms
Wall time: 144 ms
